In [1]:
import requests
import base64
import os
import json

In [2]:
"""
# Я знаю, что токены не стоить хранить на GitHub, Вы можете ознакомиться с тем, как я обычно делаю это в репозиторииях
# https://github.com/Eugene-Fed/OZON-MoySklad_server и 
# https://github.com/Eugene-Fed/mp_stats_loader
# К тому времени когда репозиторий будет открыт - ключи уже перестанут действовать,
# поэтому я уверен в безопасности своего аккаунта
"""

# Список всех настроечных параметров/констант
OAUTH_TOKEN = '<OAuth_token_by_Yandex_ID>'     # OAuth токен аккаунта Яндекс
FOLDER_ID = '<folder_id_from_yandex_cloud>'       # Идентификатор рабочего каталога Я.Cloud
WORK_DIR = 'pass_photos'
GET_AIM_TOKEN_URL = 'https://iam.api.cloud.yandex.net/iam/v1/tokens'
GET_IMAGE_RECOGNITION_URL = 'https://vision.api.cloud.yandex.net/vision/v1/batchAnalyze'
LANG = "ru"
DOC_TYPE = "passport"

### Обмениваем OAuth-токен аккаунта Яндекс на IAM-токен

In [3]:
data = {
    'yandexPassportOauthToken': OAUTH_TOKEN
}
 
response = requests.post(url=GET_AIM_TOKEN_URL, json=data)
print(response)
iam_token = response.json()['iamToken']

<Response [200]>


### Вынес некоторый код в функции для удобства, чтобы потом в цикле пройтись по всем изображениям

In [4]:
# Функция на основе инструкции Яндекс.Cloud
def encode_file(file_name: str) -> bytearray:             # Кодируем изображение в байтовую строку
    # Яндекс в своей инструкции забыл про параметр 'rb', без которого невозможно прочитать изображение
    with open(file_name, 'rb') as f:        
        file_content = f.read()
        return base64.b64encode(file_content)
    
# Функция для получения списка файлов из каталога с фотографиями (как в task_1)
def get_files(directory: str) -> list:
    names = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpeg") or filename.endswith(".jpg") or filename.endswith(".png"):
            names.append(os.path.join(directory, filename))

    return names

In [5]:
# Переводим полученный кусок JSON в читаемый текст
def parse_data(pass_data: json) -> str:
    # Яндекс выдает нам паспортные данные как список словарей. Поэтому необходимо пройтись по списку и
    # передать значение в parsed_data согласно его ключу.
    
    print(pass_data)
    parsed_data = {'issued_by': '', 'expiration_date': '', 'subdivision': '', 'surname': '', 'name': '',
                  'middle_name': '', 'gender': '', 'birth_date': '', 'birth_place': '', 'number': ''}
    
    # Идем по всем элементам полученного JSON-списка и по соответствующему полю 'name' передаем в наш словарь значение 'text'
    for item in pass_data:
        #TODO найти более изящный способ получить element через map/zip или т.п.
        element = dict([tuple(item.values())])    # {'name': 'ключ', 'text': 'значени'} --> {'ключ': 'значение'}
        parsed_data.update(element) # Забираем значение по ключу. Если такого ключа нет - он добавится в словарь.
    
    # Но для вывода мы используем только те ключи, которые нам необходимы
    result_text = f"Кем выдан: {parsed_data['issued_by']}\nДата выдачи: {parsed_data['expiration_date']}\n"
    result_text += f"Код подразделения: {parsed_data['subdivision']}\nНомер паспорта: {parsed_data['number']}\n"
    result_text += f"Фамилия: {parsed_data['surname']}\nИмя: {parsed_data['name']}\nОтчество: {parsed_data['middle_name']}\n"
    result_text += f"Пол: {parsed_data['gender']}\nДата рождения: {parsed_data['birth_date']}\n"
    result_text += f"Место Рождения: {parsed_data['birth_place']}\n\n"
    print(result_text)
    
    return(result_text)

### Отправляем запросы к Яндексу

In [6]:
files = get_files(WORK_DIR)    # Получаем список изображений в рабочей папке
headers = {'Authorization': f'Bearer {iam_token}', 'Content-Type': 'application/json'}

# Создаем пустой файл, в который в цикле будем добавлять данные паспартов после каждого запроса к API
with open('task_2.txt', 'w') as f:
    pass

# проходим по всем изображениям, получаем для каждого представление в byte64 и отправляем запрос
for file in files:
    body_json = {
        "folderId": FOLDER_ID,
        "analyze_specs": [{
            "content": str(encode_file(file))[2:-1],  # про то, что byte64 невозможно сериализовать в JSON Яндекс тоже забыли
            "features": [{
                "type": "TEXT_DETECTION",
                "text_detection_config": {
                    "language_codes": [LANG],
                    "model": DOC_TYPE
                }
            }]
        }]
    }

    response = requests.post(url=GET_IMAGE_RECOGNITION_URL, headers=headers, json=body_json)
    print(response)

    with open('task_2.txt', 'a') as f:
        f.write(f"========IMAGE '{file}' ========\n")
        f.write(parse_data(response.json()['results'][0]['results'][0]['textDetection']['pages'][0]['entities']))

<Response [200]>
[{'name': 'citizenship', 'text': 'rus'}, {'name': 'expiration_date', 'text': '-'}, {'name': 'gender', 'text': 'муж'}, {'name': 'subdivision', 'text': '772-112'}, {'name': 'issued_by', 'text': 'паспортно-визовым отделением овд пресненского района увд цао г. москвы'}, {'name': 'surname', 'text': 'трамп'}, {'name': 'name', 'text': 'дональд'}, {'name': 'middle_name', 'text': 'джон'}, {'name': 'birth_date', 'text': '14.06.1946'}, {'name': 'birth_place', 'text': 'гор. куинс штат нью-йорк сша'}, {'name': 'number', 'text': '4507123456'}]
Кем выдан: паспортно-визовым отделением овд пресненского района увд цао г. москвы
Дата выдачи: -
Код подразделения: 772-112
Номер паспорта: 4507123456
Фамилия: трамп
Имя: дональд
Отчество: джон
Пол: муж
Дата рождения: 14.06.1946
Место Рождения: гор. куинс штат нью-йорк сша


<Response [200]>
[{'name': 'expiration_date', 'text': '-'}, {'name': 'citizenship', 'text': 'rus'}, {'name': 'gender', 'text': 'жен'}, {'name': 'issued_by', 'text': 'мро у